In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from keras.models import Sequential
# from deel.lip.model import Sequential
from deel.lip.activations import GroupSort
from deel.lip.losses import MulticlassHKR, MulticlassKR, HKR, HingeMargin
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
import keras.ops as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [3]:
# first we select the two classes
selected_classes = [0, 8]  # must be two classes as we perform binary classification


def prepare_data(x, y, class_a=0, class_b=8):
    """
    This function convert the MNIST data to make it suitable for our binary classification
    setup.
    """
    # select items from the two selected classes
    mask = (y == class_a) + (
        y == class_b
    )  # mask to select only items from class_a or class_b
    x = x[mask]
    y = y[mask]
    x = x.astype("float32")
    y = y.astype("float32")
    # convert from range int[0,255] to float32[-1,1]
    x /= 255
    x = x.reshape((-1, 28, 28, 1))
    # change label to binary classification {-1,1}
    y[y == class_a] = 1.0
    y[y == class_b] = 0.0
    return x, y.reshape((-1, 1))


# now we load the dataset
(x_train, y_train_ord), (x_test, y_test_ord) = mnist.load_data()
# prepare the data
x_train, y_train = prepare_data(
    x_train, y_train_ord, selected_classes[0], selected_classes[1]
)
x_test, y_test = prepare_data(
    x_test, y_test_ord, selected_classes[0], selected_classes[1]
)
y_test_ord = y_test[:,0]
y_train_ord = y_train[:,0]
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [4]:
x_train = np.transpose(x_train,(0,3,1,2))
x_test = np.transpose(x_test,(0,3,1,2))

In [5]:
vanilla_model = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras")
vanilla_model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)
vanilla_model.summary()

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "lipModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_2 (Dense)        │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,664 (100.25 KB)

 Trainable params: 25,664 (100.25 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
vanilla_model_bis = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")
vanilla_model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)
vanilla_model_bis.summary()

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (10, 784)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (10, 32)               │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (10, 32)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (10, 16)               │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (10, 16)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (10, 4)                │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,716 (100.45 KB)

 Trainable params: 25,716 (100.45 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# strategy: first
# we select a sample from each class.
images_list = []
labels_list = []
# select only a few element from the test set
# selected = np.random.choice(len(y_test_ord), 500)
sub_y_test_ord = y_test_ord[:400]
sub_x_test = x_test[:400]
# drop misclassified elements
misclassified_mask = K.equal(
    K.argmax(vanilla_model_bis.predict(sub_x_test, verbose=0), axis=-1), sub_y_test_ord
)
sub_x_test = sub_x_test[misclassified_mask.detach().cpu().numpy()]
sub_y_test_ord = sub_y_test_ord[misclassified_mask.detach().cpu().numpy()]
for i in range(2):
    # select the 20 firsts elements of the ith label
    label_mask = sub_y_test_ord == i
    x = sub_x_test[label_mask][:100]
    y = sub_y_test_ord[label_mask][:100]
    # convert it to tensor for use with foolbox
    images = K.convert_to_tensor(x.astype("float32"), dtype="float32")
    labels = K.convert_to_tensor(y, dtype="int64")
    # repeat the input 10 times, one per misclassification target
    for j in range(100):
        images_list.append(images[j])
        labels_list.append(labels[j])
images = K.convert_to_tensor(images_list)
labels = K.convert_to_tensor(labels_list)
labels.shape

torch.Size([200])

In [13]:
def compute_binary_certificate(images, model, L=1):    
    values = model(images)[:,0]
    certificates = np.abs(values.detach().cpu().numpy())/L
    return certificates   

def compute_certificate(images, model, L=1):    
    values, _ = K.top_k(model(images), k=2)
    certificates = (values[:, 0] - values[:, 1]) / (np.sqrt(2)*L)
    return certificates.detach().cpu().numpy()   

In [9]:
lip_radius = compute_binary_certificate(images, vanilla_model)

In [19]:
import keras.ops as K
import matplotlib.pyplot as plt
import torchattacks
import torch
import torch.nn as nn
import torchattacks
from robustbench.utils import clean_accuracy

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
atk = torchattacks.FGSM(vanilla_model_bis,eps=1)
adv_image_bis = atk(images[:1], labels[:1])


In [34]:
K.argmax(vanilla_model_bis.predict(adv_image_bis[:1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


tensor(1, device='cuda:0', dtype=torch.int32)

In [35]:
vanilla_model.predict(images[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[-1.6612895]], dtype=float32)

In [32]:
atk = torchattacks.AutoAttack(vanilla_model_bis, norm='L2', eps=2.5, version="standard", n_classes=2, verbose=True, seed=1)
adv_image_2 = atk(images[:1], labels[:1])

-------------------------- running L2-attack with epsilon 2.5000 --------------------------
initial accuracy: 100.00%
parameters:  10 2 1 1
iteration: 0 - Best loss: 0.046096
iteration: 1 - Best loss: 1.089687
iteration: 2 - Best loss: 1.478883
iteration: 3 - Best loss: 1.478883
iteration: 4 - Best loss: 1.478883
iteration: 5 - Best loss: 1.478883
iteration: 6 - Best loss: 1.478883
iteration: 7 - Best loss: 1.480214
iteration: 8 - Best loss: 1.480214
iteration: 9 - Best loss: 1.484007
restart 0 - robust accuracy: 0.00% - cum. time: 0.1 s


In [36]:
K.argmax(vanilla_model_bis.predict(adv_image_2[:1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


tensor(1, device='cuda:0', dtype=torch.int32)

In [37]:
vanilla_model.predict(adv_image_2[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.61493695]], dtype=float32)